In [2]:
import nltk as nltk
nltk.download('stopwords')
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import warnings 
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/priyankachaganti/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Load data from the train data file 
train_path = 'train_file.dat'
train = np.genfromtxt(train_path, dtype=None, delimiter='\t', encoding=None)
train_data = pd.DataFrame(train).rename(columns={'f0': 'Sentiment', 'f1': 'Review'})
train_len = len(train)
train_data


Sentiment                                             Review
0              1  This book is such a life saver.  It has been s...
1              1  I bought this a few times for my older son and...
2              1  This is great for basics, but I wish the space...
3              1  This book is perfect!  I'm a first time new mo...
4              1  During your postpartum stay at the hospital th...
...          ...                                                ...
18501         -1  I really liked this monitor at first, but the ...
18502         -1  Apparently you get what you pay for.  I've use...
18503         -1  The old saying holds true with this product --...
18504         -1  We did a great deal of research before purchas...
18505         -1  I ordered these after having great success wit...

[18506 rows x 2 columns]

In [4]:
# Load data from the test data file 
test_path = 'test.dat'
test = open(test_path, 'r')
test = test.readlines()
test_data = pd.DataFrame(test)
test_data

0
0      Perfect for new parents. We were able to keep ...
1      Helps me know exactly how my babies day has go...
2      I wanted an alternative to printing out daily ...
3      My 3 month old son spend half of his days with...
4      The Baby Tracker brand books are the absolute ...
...                                                  ...
18501  WTF. The pieces don't fit together, the instru...
18502  I've gone through a couple of video baby monit...
18503  This monitor is cheap and doesn't work well. O...
18504  These monitors do not work at all, I even atte...
18505  Short story, I was very disappointed with the ...

[18506 rows x 1 columns]

def remove_html(data):
    remhtml = BeautifulSoup(data, "html.parser")
    return remhtml.get_text()

train_html = train_data['Review'].apply(remove_html)
print(train_html)

test_html = test_data[0].apply(remove_html)
print(test_html)

In [5]:
# The function coverts all the data in the files to lower case
def convert_lower(data):
    data = data.apply(lambda x: " ".join(x.lower() for x in x.split()))
    return data

train_lower_html = convert_lower(train_data['Review'])
print(train_lower_html)

test_lower_html = convert_lower(test_data[0])
print(test_lower_html)

0        this book is such a life saver. it has been so...
1        i bought this a few times for my older son and...
2        this is great for basics, but i wish the space...
3        this book is perfect! i'm a first time new mom...
4        during your postpartum stay at the hospital th...
                               ...                        
18501    i really liked this monitor at first, but the ...
18502    apparently you get what you pay for. i've used...
18503    the old saying holds true with this product --...
18504    we did a great deal of research before purchas...
18505    i ordered these after having great success wit...
Name: Review, Length: 18506, dtype: object
0        perfect for new parents. we were able to keep ...
1        helps me know exactly how my babies day has go...
2        i wanted an alternative to printing out daily ...
3        my 3 month old son spend half of his days with...
4        the baby tracker brand books are the absolute ...
             

In [6]:
# The function removes all kinds of punctuations on the text
def remove_punctuations(data):
    data = data.str.replace('[^\w\s]', '', regex=True)
    return data

train_lower_ht_punc = remove_punctuations(train_lower_html)
print(train_lower_ht_punc)

test_lower_ht_punc = remove_punctuations(test_lower_html)
print(test_lower_ht_punc)

0        this book is such a life saver it has been so ...
1        i bought this a few times for my older son and...
2        this is great for basics but i wish the space ...
3        this book is perfect im a first time new mom a...
4        during your postpartum stay at the hospital th...
                               ...                        
18501    i really liked this monitor at first but the s...
18502    apparently you get what you pay for ive used p...
18503    the old saying holds true with this product  y...
18504    we did a great deal of research before purchas...
18505    i ordered these after having great success wit...
Name: Review, Length: 18506, dtype: object
0        perfect for new parents we were able to keep t...
1        helps me know exactly how my babies day has go...
2        i wanted an alternative to printing out daily ...
3        my 3 month old son spend half of his days with...
4        the baby tracker brand books are the absolute ...
             

In [7]:
#The function removes all the contractions from the data
def interpret(text):
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = ' '.join(word for word in text.split() if len(word)>=3)
    return text

In [8]:
#The function removes stopwords from the data.
stop_words = set(stopwords.words('english'))
stop_words.remove("no")
stop_words.remove("not")
# print(stop_words)
def nlp_preprocessing(total_text):
    if type(total_text) is not int:
        string = ""
        total_text=interpret(total_text)
    return total_text

stop_words_remove_train_data=[]
for data in train_lower_ht_punc:
    stop_words_remove_train_data.append(nlp_preprocessing(data))

In [9]:
stop_words_remove_test_data = []
for data in test_lower_ht_punc:
    stop_words_remove_test_data.append(nlp_preprocessing(data))

In [10]:
print(len(stop_words_remove_train_data))

18506


In [11]:
print(len(stop_words_remove_test_data))

18506


In [12]:
#This function reduces words to it't root
def stem_check(data):
    ps = nltk.porter.PorterStemmer() 
    data = ' '.join([ps.stem(word) for word in data.split()]) 
    return data

train_stem_output =[stem_check(data) for data in stop_words_remove_train_data]

print(train_stem_output[0])

thi book such life saver ha been help abl back track trend answer pediatrician question commun with each other when you are differ time the night with newborn think one those thing that everyon should requir have befor they leav the hospit went through all the page the newborn version then move the infant version and will finish the second infant book third total right our babi turn see other thing that are must have for babi


In [13]:
test_stem_output =list(map(stem_check,stop_words_remove_test_data))

In [14]:
vectorizer = CountVectorizer(ngram_range=(1,2),max_features=10000)
train_vectorizer = vectorizer.fit_transform(train_stem_output)

In [15]:
test_vectorizer = vectorizer.transform(test_stem_output)

In [16]:
print(len(train_vectorizer.toarray()))

18506


In [17]:
# Logistic Regression Implementation
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
distributions = dict(C=uniform(loc=0, scale=4),
                      penalty=['l2'])
regressor  = LogisticRegression(solver='lbfgs',max_iter=100)

In [18]:
clf = RandomizedSearchCV(regressor, distributions, random_state=0)

In [19]:
clf.fit(train_vectorizer,train_data['Sentiment'])

RandomizedSearchCV(estimator=LogisticRegression(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb102c73850>,
                                        'penalty': ['l2']},
                   random_state=0)

In [20]:
clf.best_params_

{'C': 2.4110535042865755, 'penalty': 'l2'}

In [21]:
regressor  = LogisticRegression(solver='lbfgs',max_iter=100,C = 2.4110535042865755)

In [22]:
regressor.fit(train_vectorizer,train_data['Sentiment'])

LogisticRegression(C=2.4110535042865755)

In [23]:
regressor.score(train_vectorizer,train_data['Sentiment'])

0.9969199178644764

In [24]:
test_output=regressor.predict(test_vectorizer)

fptr = open("./Predictions.dat", 'w')
for prediction in test_output:
    if prediction > 0:
        fptr.write('+1\n')
    else:
        fptr.write('-1\n')
fptr.close()